In [1]:
%pip install requests beautifulsoup4 lxml python-dateutil feedparser

  Using cached feedparser-6.0.12-py3-none-any.whl.metadata (2.7 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached feedparser-6.0.12-py3-none-any.whl (81 kB)
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6104 sha256=03c8105bf184a641c4c6a04fa83f3f06ec5cbbd3348b1f8d66ac149bc2d9c17e
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\f0\69\93\a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k

   ---------------------------------------- 2/2 [feedparser]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import csv
import os
import re
import time
import hashlib
from datetime import datetime, timezone, timedelta
from typing import Optional, Dict, Any, List, Set, Tuple

import requests
import feedparser
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

VN_TZ = timezone(timedelta(hours=7))

# ================== CONFIG ==================
# Danh sách các RSS feeds từ các báo Việt Nam
RSS_FEEDS = [
    # VnExpress RSS (15 feeds)
    "https://vnexpress.net/rss/tin-moi-nhat.rss",
    "https://vnexpress.net/rss/thoi-su.rss",
    "https://vnexpress.net/rss/the-gioi.rss",
    "https://vnexpress.net/rss/kinh-doanh.rss",
    "https://vnexpress.net/rss/giai-tri.rss",
    "https://vnexpress.net/rss/the-thao.rss",
    "https://vnexpress.net/rss/phap-luat.rss",
    "https://vnexpress.net/rss/giao-duc.rss",
    "https://vnexpress.net/rss/suc-khoe.rss",
    "https://vnexpress.net/rss/gia-dinh.rss",
    "https://vnexpress.net/rss/du-lich.rss",
    "https://vnexpress.net/rss/khoa-hoc.rss",
    "https://vnexpress.net/rss/so-hoa.rss",
    "https://vnexpress.net/rss/oto-xe-may.rss",
    "https://vnexpress.net/rss/y-kien.rss",
    
    # Dân Trí RSS (10 feeds)
    "https://dantri.com.vn/rss/trang-chu.rss",
    "https://dantri.com.vn/rss/xa-hoi.rss",
    "https://dantri.com.vn/rss/the-gioi.rss",
    "https://dantri.com.vn/rss/kinh-doanh.rss",
    "https://dantri.com.vn/rss/the-thao.rss",
    "https://dantri.com.vn/rss/giai-tri.rss",
    "https://dantri.com.vn/rss/giao-duc.rss",
    "https://dantri.com.vn/rss/suc-khoe.rss",
    "https://dantri.com.vn/rss/du-lich.rss",
    "https://dantri.com.vn/rss/o-to-xe-may.rss",
    
    # Tuổi Trẻ RSS (9 feeds)
    "https://tuoitre.vn/rss/tin-moi-nhat.rss",
    "https://tuoitre.vn/rss/thoi-su.rss",
    "https://tuoitre.vn/rss/the-gioi.rss",
    "https://tuoitre.vn/rss/phap-luat.rss",
    "https://tuoitre.vn/rss/kinh-doanh.rss",
    "https://tuoitre.vn/rss/giao-duc.rss",
    "https://tuoitre.vn/rss/the-thao.rss",
    "https://tuoitre.vn/rss/giai-tri.rss",
    "https://tuoitre.vn/rss/xe.rss",
    
    # Thanh Niên RSS (8 feeds)
    "https://thanhnien.vn/rss/home.rss",
    "https://thanhnien.vn/rss/thoi-su.rss",
    "https://thanhnien.vn/rss/the-gioi.rss",
    "https://thanhnien.vn/rss/kinh-te.rss",
    "https://thanhnien.vn/rss/van-hoa.rss",
    "https://thanhnien.vn/rss/the-thao.rss",
    "https://thanhnien.vn/rss/cong-nghe.rss",
    "https://thanhnien.vn/rss/gioi-tre.rss",
    
    # VietnamNet RSS (5 feeds)
    "https://vietnamnet.vn/rss/thoi-su.rss",
    "https://vietnamnet.vn/rss/the-gioi.rss",
    "https://vietnamnet.vn/rss/kinh-doanh.rss",
    "https://vietnamnet.vn/rss/giao-duc.rss",
    "https://vietnamnet.vn/rss/the-thao.rss",
    
    # Lao Động RSS (11 feeds)
    "https://laodong.vn/rss/home.rss",
    "https://laodong.vn/rss/cong-doan.rss",
    "https://laodong.vn/rss/xa-hoi.rss",
    "https://laodong.vn/rss/kinh-doanh.rss",
    "https://laodong.vn/rss/van-hoa-giai-tri.rss",
    "https://laodong.vn/rss/xe.rss",
    "https://laodong.vn/rss/thoi-su.rss",
    "https://laodong.vn/rss/the-gioi.rss",
    "https://laodong.vn/rss/phap-luat.rss",
    "https://laodong.vn/rss/the-thao.rss",
    "https://laodong.vn/rss/suc-khoe.rss",
    
    # Người Lao Động RSS (17 feeds)
    "https://nld.com.vn/rss/home.rss",
    "https://nld.com.vn/rss/thoi-su.rss",
    "https://nld.com.vn/rss/quoc-te.rss",
    "https://nld.com.vn/rss/lao-dong.rss",
    "https://nld.com.vn/rss/ban-doc.rss",
    "https://nld.com.vn/rss/net-zero.rss",
    "https://nld.com.vn/rss/kinh-te.rss",
    "https://nld.com.vn/rss/suc-khoe.rss",
    "https://nld.com.vn/rss/giao-duc-khoa-hoc.rss",
    "https://nld.com.vn/rss/phap-luat.rss",
    "https://nld.com.vn/rss/van-hoa-van-nghe.rss",
    "https://nld.com.vn/rss/giai-tri.rss",
    "https://nld.com.vn/rss/the-thao.rss",
    "https://nld.com.vn/rss/ai-365.rss",
    "https://nld.com.vn/rss/du-lich-xanh.rss",
    "https://nld.com.vn/rss/khoa-hoc.rss",
    "https://nld.com.vn/rss/nguoi-lao-dong-news.rss",
    
    # VietnamPlus RSS (19 feeds)
    "https://www.vietnamplus.vn/rss/home.rss",
    "https://www.vietnamplus.vn/rss/chinhtri-291.rss",
    "https://www.vietnamplus.vn/rss/thegioi-209.rss",
    "https://www.vietnamplus.vn/rss/thegioi/asean-356.rss",
    "https://www.vietnamplus.vn/rss/thegioi/chaua-tbd-352.rss",
    "https://www.vietnamplus.vn/rss/thegioi/trungdong-230.rss",
    "https://www.vietnamplus.vn/rss/thegioi/chauau-354.rss",
    "https://www.vietnamplus.vn/rss/thegioi/chauphi-357.rss",
    "https://www.vietnamplus.vn/rss/thegioi/chaumy-355.rss",
    "https://www.vietnamplus.vn/rss/kinhte-311.rss",
    "https://www.vietnamplus.vn/rss/kinhte/kinhdoanh-342.rss",
    "https://www.vietnamplus.vn/rss/kinhte/taichinh-343.rss",
    "https://www.vietnamplus.vn/rss/xahoi-314.rss",
    "https://www.vietnamplus.vn/rss/xahoi/giaoduc-316.rss",
    "https://www.vietnamplus.vn/rss/xahoi/yte-325.rss",
    "https://www.vietnamplus.vn/rss/xahoi/phapluat-327.rss",
    "https://www.vietnamplus.vn/rss/xahoi/giaothong-358.rss",
    "https://www.vietnamplus.vn/rss/doisong-320.rss",
    "https://www.vietnamplus.vn/rss/thethao-214.rss",
    
    # Soha RSS (9 feeds)
    "https://soha.vn/rss/home.rss",
    "https://soha.vn/rss/thoi-su-xa-hoi.rss",
    "https://soha.vn/rss/kinh-doanh.rss",
    "https://soha.vn/rss/quoc-te.rss",
    "https://soha.vn/rss/the-thao.rss",
    "https://soha.vn/rss/giai-tri.rss",
    "https://soha.vn/rss/phap-luat.rss",
    "https://soha.vn/rss/viet-nam-vuon-minh.rss",
    "https://soha.vn/rss/sea-games-32.rss",
    
    # Nhân Dân RSS (16 feeds)
    "https://nhandan.vn/rss/home.rss",
    "https://nhandan.vn/rss/chinhtri-1171.rss",
    "https://nhandan.vn/rss/xa-luan-1176.rss",
    "https://nhandan.vn/rss/xay-dung-dang-1179.rss",
    "https://nhandan.vn/rss/kinhte-1185.rss",
    "https://nhandan.vn/rss/chungkhoan-1191.rss",
    "https://nhandan.vn/rss/phapluat-1287.rss",
    "https://nhandan.vn/rss/du-lich-1257.rss",
    "https://nhandan.vn/rss/thegioi-1231.rss",
    "https://nhandan.vn/rss/asean-704471.rss",
    "https://nhandan.vn/rss/chau-phi-704476.rss",
    "https://nhandan.vn/rss/chau-my-704475.rss",
    "https://nhandan.vn/rss/chau-au-704474.rss",
    "https://nhandan.vn/rss/trung-dong-704473.rss",
    "https://nhandan.vn/rss/chau-a-tbd-704472.rss",
    "https://nhandan.vn/rss/thethao-1224.rss",
    
    # Báo Tin Tức RSS (10 feeds)
    "https://baotintuc.vn/tin-moi-nhat.rss",
    "https://baotintuc.vn/thoi-su.rss",
    "https://baotintuc.vn/the-gioi.rss",
    "https://baotintuc.vn/kinh-te.rss",
    "https://baotintuc.vn/xa-hoi.rss",
    "https://baotintuc.vn/phap-luat.rss",
    "https://baotintuc.vn/giao-duc.rss",
    "https://baotintuc.vn/van-hoa.rss",
    "https://baotintuc.vn/the-thao.rss",
    "https://baotintuc.vn/quan-su.rss",
    
    # Kiến Thức RSS (8 feeds)
    "https://kienthuc.net.vn/rss/home.rss",
    "https://kienthuc.net.vn/rss/nha-khoa-hoc-345.rss",
    "https://kienthuc.net.vn/rss/spotlight-379.rss",
    "https://kienthuc.net.vn/rss/chinh-tri-348.rss",
    "https://kienthuc.net.vn/rss/xa-hoi-349.rss",
    "https://kienthuc.net.vn/rss/the-gioi-350.rss",
    "https://kienthuc.net.vn/rss/quan-su-359.rss",
    "https://kienthuc.net.vn/rss/giai-tri-365.rss",
]

# Crawl tất cả bài viết từ RSS feeds
END_DATE = "2026-01-15"  # YYYY-MM-DD - chỉ lấy bài >= ngày này

CSV_PATH = "rss_feed_articles_v2.csv"

# Có fetch full content từ URL gốc không (chậm hơn nhưng đầy đủ hơn)
FETCH_FULL_CONTENT = True

TIMEOUT = 25
REQUEST_DELAY_BASE = 0.25
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; RSSCrawler/1.0)",
    "Accept": "application/rss+xml, application/xml, text/xml, */*",
}
# ===========================================

CSV_HEADER = [
    "id",
    "title",
    "published_at",        # ISO UTC
    "source.name",
    "url",
    "language",
    "category.primary",
    "keywords",
    "entities",
    "content.text",
]

SOURCE_NAME = "RSS_Feed"
DEFAULT_LANGUAGE = "vi"
DEBUG = False

# ----- HTTP session with retry -----
session = requests.Session()
session.headers.update(HEADERS)

retry = Retry(
    total=6,
    connect=6,
    read=6,
    backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET", "HEAD"],
    respect_retry_after_header=True,
    raise_on_status=False,
)
adapter = HTTPAdapter(max_retries=retry, pool_connections=50, pool_maxsize=50)
session.mount("http://", adapter)
session.mount("https://", adapter)


def log(msg: str):
    if DEBUG:
        print(msg)


def polite_sleep():
    time.sleep(REQUEST_DELAY_BASE)


def md5_id(text: str) -> str:
    return hashlib.md5(text.encode("utf-8")).hexdigest()


def fetch_text(url: str) -> str:
    r = session.get(url, timeout=TIMEOUT, allow_redirects=True)
    r.raise_for_status()
    return r.text


def ensure_csv_header(csv_path: str):
    if not os.path.exists(csv_path) or os.path.getsize(csv_path) == 0:
        with open(csv_path, "w", encoding="utf-8", newline="") as f:
            csv.writer(f).writerow(CSV_HEADER)


def load_seen_from_csv(csv_path: str) -> Tuple[Set[str], Set[str]]:
    seen_urls, seen_ids = set(), set()
    if not os.path.exists(csv_path):
        return seen_urls, seen_ids
    try:
        with open(csv_path, "r", encoding="utf-8", newline="") as f:
            r = csv.reader(f)
            header = next(r, None)
            if not header:
                return seen_urls, seen_ids
            id_idx = header.index("id") if "id" in header else 0
            url_idx = header.index("url") if "url" in header else 4
            for row in r:
                if len(row) > url_idx:
                    u = row[url_idx].strip()
                    if u:
                        seen_urls.add(u)
                if len(row) > id_idx:
                    i = row[id_idx].strip()
                    if i:
                        seen_ids.add(i)
    except Exception:
        pass
    return seen_urls, seen_ids


def append_row(csv_path: str, row: Dict[str, Any]):
    with open(csv_path, "a", encoding="utf-8", newline="") as f:
        w = csv.writer(f)
        w.writerow([row.get(k, "") for k in CSV_HEADER])
        f.flush()


def iso_to_local_date(iso_utc: str) -> Optional[str]:
    if not iso_utc:
        return None
    try:
        dt = datetime.fromisoformat(iso_utc.replace("Z", "+00:00"))
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt.astimezone(VN_TZ).date().isoformat()
    except Exception:
        return None


def parse_rss_date(date_str: str) -> Optional[str]:
    """
    Parse RSS date format to ISO UTC
    RSS thường dùng RFC 2822 hoặc ISO format
    """
    if not date_str:
        return None
    
    try:
        # feedparser tự động parse date
        from email.utils import parsedate_to_datetime
        dt = parsedate_to_datetime(date_str)
        return dt.astimezone(timezone.utc).isoformat()
    except Exception:
        pass
    
    # Thử ISO format
    try:
        dt = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=VN_TZ)
        return dt.astimezone(timezone.utc).isoformat()
    except Exception:
        pass
    
    return None


def extract_category_from_url(url: str) -> Optional[str]:
    """Trích xuất category từ URL pattern"""
    from urllib.parse import urlparse
    
    # Mapping các patterns phổ biến
    category_map = {
        'thoi-su': 'Thời sự',
        'the-gioi': 'Thế giới', 
        'xa-hoi': 'Xã hội',
        'kinh-doanh': 'Kinh doanh',
        'giai-tri': 'Giải trí',
        'the-thao': 'Thể thao',
        'phap-luat': 'Pháp luật',
        'giao-duc': 'Giáo dục',
        'suc-khoe': 'Sức khỏe',
        'gia-dinh': 'Gia đình',
        'du-lich': 'Du lịch',
        'khoa-hoc': 'Khoa học',
        'so-hoa': 'Số hóa',
        'cong-nghe': 'Công nghệ',
        'oto-xe-may': 'Ôtô-Xe máy',
        'doi-song': 'Đời sống',
        'van-hoa': 'Văn hóa',
        'tin-tuc': 'Tin tức',
        'video': 'Video',
    }
    
    try:
        parsed = urlparse(url)
        path_parts = [p for p in parsed.path.split('/') if p]
        
        # Tìm category trong các phần của URL (chỉ xét 2 phần đầu)
        for part in path_parts[:2]:
            for pattern, category in category_map.items():
                if pattern in part:
                    return category
    except Exception:
        pass
    
    return None


def extract_keywords_from_entry(entry: Any, url: str) -> List[str]:
    """Trích xuất keywords từ RSS entry và URL"""
    keywords = []
    
    # 1. Lấy từ tags RSS (Dân Trí, Tuổi Trẻ có field này)
    if 'tags' in entry and entry.tags:
        for tag in entry.tags:
            term = tag.get('term', '').strip()
            if term and term not in keywords:
                keywords.append(term)
    
    # 2. Lấy từ category field (nếu không có trong tags)
    if 'category' in entry and entry.category:
        cat = entry.category.strip()
        if cat and cat not in keywords:
            keywords.append(cat)
    
    # 3. Trích xuất từ URL
    url_category = extract_category_from_url(url)
    if url_category and url_category not in keywords:
        keywords.append(url_category)
    
    return keywords


def extract_content_from_html(html_content: str) -> str:
    """Trích xuất text từ HTML content trong RSS"""
    if not html_content:
        return ""
    
    try:
        soup = BeautifulSoup(html_content, "lxml")
        # Lấy tất cả text, loại bỏ tags
        text = soup.get_text(separator=" ", strip=True)
        # Làm sạch whitespace
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    except Exception:
        return html_content


def fetch_article_content(url: str) -> str:
    """
    Fetch nội dung đầy đủ từ URL bài viết
    Nếu RSS chỉ có summary, cần fetch trang gốc
    """
    try:
        html = fetch_text(url)
        soup = BeautifulSoup(html, "lxml")
        
        # Thử các selector phổ biến
        article_body = None
        selectors = [
            "article",
            ".article-content",
            ".post-content",
            ".entry-content",
            ".content",
            "main",
        ]
        
        for selector in selectors:
            article_body = soup.select_one(selector)
            if article_body:
                break
        
        if article_body:
            paragraphs = article_body.find_all("p")
            text_parts = []
            for p in paragraphs:
                text = p.get_text(strip=True)
                if text:
                    text_parts.append(text)
            return " ".join(text_parts)
        
        return ""
    except Exception as e:
        log(f"Failed to fetch article content from {url}: {e}")
        return ""


def parse_rss_entry(entry: Any, seen_urls: Set[str], fetch_full_content: bool = False) -> Optional[Dict[str, Any]]:
    """Parse một entry từ RSS feed"""
    
    # URL
    url = entry.get("link", "").strip()
    if not url or url in seen_urls:
        return None
    
    # Title
    title = entry.get("title", "").strip()
    
    # Published date
    pub = ""
    if hasattr(entry, "published_parsed") and entry.published_parsed:
        try:
            dt = datetime(*entry.published_parsed[:6], tzinfo=timezone.utc)
            pub = dt.isoformat()
        except Exception:
            pass
    
    if not pub and "published" in entry:
        pub = parse_rss_date(entry.published) or ""
    
    if not pub and "pubDate" in entry:
        pub = parse_rss_date(entry.pubDate) or ""
    
    # Content
    content_text = ""
    
    # Thử lấy content từ RSS
    if "content" in entry and entry.content:
        # feedparser trả về list
        content_html = entry.content[0].get("value", "") if isinstance(entry.content, list) else entry.content
        content_text = extract_content_from_html(content_html)
    
    # Nếu không có content, thử summary/description
    if not content_text:
        if "summary" in entry:
            content_text = extract_content_from_html(entry.summary)
        elif "description" in entry:
            content_text = extract_content_from_html(entry.description)
    
    # Nếu cần fetch full content từ URL gốc
    if fetch_full_content and url:
        full_content = fetch_article_content(url)
        if full_content and len(full_content) > len(content_text):
            content_text = full_content
    
    # Keywords - tự động trích xuất từ tags RSS hoặc URL
    keywords = extract_keywords_from_entry(entry, url)
    
    # Category - ưu tiên từ tags RSS, sau đó từ URL
    category = ""
    if keywords:
        category = keywords[0]  # Lấy keyword đầu tiên làm primary category
    
    # Author có thể là source
    author = entry.get("author", "")
    
    return {
        "url": url,
        "title": title,
        "published_at": pub,
        "content_text": content_text,
        "category": category,
        "keywords": keywords,
        "author": author,
    }


def make_row(data: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "id": md5_id(data["url"]),
        "title": data.get("title") or "",
        "published_at": data.get("published_at") or "",
        "source.name": data.get("author") or SOURCE_NAME,
        "url": data["url"],
        "language": DEFAULT_LANGUAGE,
        "category.primary": data.get("category") or "",
        "keywords": "|".join(data.get("keywords") or []),
        "entities": "",
        "content.text": data.get("content_text") or "",
    }


def crawl_rss_feed(feed_url: str, end_date: str, seen_urls: Set[str], seen_ids: Set[str], fetch_full_content: bool = False) -> int:
    """
    Crawl RSS feed
    fetch_full_content: True nếu muốn fetch nội dung đầy đủ từ URL gốc
    """
    added = 0
    skipped_old = 0
    skipped_duplicate = 0
    
    try:
        # Parse RSS feed
        print(f"Fetching RSS feed: {feed_url}")
        feed = feedparser.parse(feed_url)
        
        if not feed.entries:
            print("No entries found in RSS feed")
            return 0
        
        print(f"Found {len(feed.entries)} entries in RSS feed")
        
        # Parse end_date
        end_dt = datetime.fromisoformat(end_date).replace(tzinfo=VN_TZ)
        
        for entry in feed.entries:
            try:
                # Parse entry
                data = parse_rss_entry(entry, seen_urls, fetch_full_content=False)
                if not data:
                    skipped_duplicate += 1
                    continue
                
                # Filter by date
                pub_iso = data.get("published_at")
                if pub_iso:
                    pub_local_date = iso_to_local_date(pub_iso)
                    if pub_local_date and pub_local_date < end_date:
                        skipped_old += 1
                        continue
                
                # Check duplicate by ID
                aid = md5_id(data["url"])
                if aid in seen_ids:
                    skipped_duplicate += 1
                    continue
                
                # Fetch full content if needed
                if fetch_full_content:
                    full_content = fetch_article_content(data["url"])
                    if full_content and len(full_content) > len(data.get("content_text", "")):
                        data["content_text"] = full_content
                
                # Ghi bài vào CSV
                row = make_row(data)
                append_row(CSV_PATH, row)
                seen_urls.add(data["url"])
                seen_ids.add(aid)
                added += 1
                
                print(f"Added: {data.get('title', '')[:80]}")
                
                if fetch_full_content:
                    polite_sleep()
                    
            except Exception as e:
                log(f"Error processing entry: {e}")
                continue
        
        # Summary log
        print(f"\nSummary: {added} added, {skipped_duplicate} duplicates, {skipped_old} old")
        return added
        
    except Exception as e:
        print(f"Error crawling RSS feed: {e}")
        import traceback
        traceback.print_exc()
        return 0


def main():
    ensure_csv_header(CSV_PATH)
    seen_urls, seen_ids = load_seen_from_csv(CSV_PATH)
    
    total_added = 0
    
    for i, feed_url in enumerate(RSS_FEEDS, 1):
        print(f"\n{'='*80}")
        print(f"Processing RSS Feed {i}/{len(RSS_FEEDS)}")
        print(f"{'='*80}")
        
        try:
            added = crawl_rss_feed(feed_url, END_DATE, seen_urls, seen_ids, fetch_full_content=FETCH_FULL_CONTENT)
            total_added += added
            print(f"✓ Added {added} articles from this feed")
        except Exception as e:
            print(f"✗ Error with feed {feed_url}: {e}")
            continue
        
        # Delay giữa các feeds
        if i < len(RSS_FEEDS):
            time.sleep(2)
    
    print(f"\n{'='*80}")
    print(f"SUMMARY")
    print(f"{'='*80}")
    print(f"Total feeds processed: {len(RSS_FEEDS)}")
    print(f"Total articles added: {total_added}")
    print(f"Output file: {CSV_PATH}")
    print(f"{'='*80}")


if __name__ == "__main__":
    main()


Processing RSS Feed 1/137
Fetching RSS feed: https://vnexpress.net/rss/tin-moi-nhat.rss
Found 54 entries in RSS feed
Added: Tên cướp ngân hàng ném tiền vào xe rác khi tháo chạy
Added: Sắp có nghị quyết gỡ vướng Nghị định 46 về an toàn thực phẩm
Added: Ông Tập đề xuất 'kế hoạch lớn' với ông Putin để phát triển quan hệ
Added: Tôi vay 1,7 tỷ đồng mua nhà vì đi thuê mười năm mất hơn một tỷ
Added: Đường dây cung cấp ma túy liên tỉnh bị triệt phá
Added: Kẻ cướp ngân hàng bị khống chế tại hiện trường
Added: Những sai lầm khiến bệnh răng miệng dễ xuất hiện dịp Tết
Added: 200 nhà khoa học chia sẻ về xu hướng công nghệ sinh học và năng lượng
Added: Hai trùm buôn lậu 546 kg vàng bị tuyên 10 năm tù
Added: Tập đoàn Nhật Bản rót 1.900 tỷ đồng mua dự án cao ốc ở TP HCM
Added: Xe buýt đi sân bay Long Thành nên có làn riêng để đảm bảo đúng giờ'?
Added: Trung tâm tài chính, khu thương mại tự do 'hút' chú ý của FDI
Added: Nước nào có tỷ phú giàu nhất Đông Nam Á?
Added: Elon Musk thành người đầu tiên có 

In [2]:
# TEST: Kiểm tra cấu trúc RSS feed
import feedparser
import requests

rss_url = "https://rss.app/feeds/b72SnQUmvktz54SH.xml"

try:
    print(f"Fetching RSS feed: {rss_url}")
    feed = feedparser.parse(rss_url)
    
    print(f"\nFeed title: {feed.feed.get('title', 'N/A')}")
    print(f"Feed link: {feed.feed.get('link', 'N/A')}")
    print(f"Total entries: {len(feed.entries)}")
    
    if feed.entries:
        print("\n" + "="*80)
        print("First entry details:")
        print("="*80)
        entry = feed.entries[0]
        
        print(f"\nTitle: {entry.get('title', 'N/A')}")
        print(f"Link: {entry.get('link', 'N/A')}")
        print(f"Published: {entry.get('published', 'N/A')}")
        
        if hasattr(entry, 'published_parsed'):
            print(f"Published parsed: {entry.published_parsed}")
        
        if 'author' in entry:
            print(f"Author: {entry.author}")
        
        if 'tags' in entry and entry.tags:
            print(f"Tags: {[tag.get('term', '') for tag in entry.tags]}")
        
        # Content
        if 'content' in entry:
            content = entry.content[0].get('value', '') if isinstance(entry.content, list) else entry.content
            print(f"\nContent length: {len(content)} chars")
            print(f"Content preview: {content[:300]}...")
        
        if 'summary' in entry:
            print(f"\nSummary length: {len(entry.summary)} chars")
            print(f"Summary preview: {entry.summary[:300]}...")
        
        print("\n" + "="*80)
        print(f"Showing titles of first 10 entries:")
        print("="*80)
        for i, e in enumerate(feed.entries[:10], 1):
            print(f"{i}. {e.get('title', 'No title')[:100]}")
            
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


Fetching RSS feed: https://rss.app/feeds/b72SnQUmvktz54SH.xml

Feed title: Việt Nam - Google Tin tức
Feed link: https://news.google.com/topics/CAAqIggKIhxDQkFTRHdvSkwyMHZNREZqY21RMUVnSjJhU2dBUAE?hl=vi&gl=VN&ceid=VN%3Avi
Total entries: 25

First entry details:

Title: Không có KPI cho người đứng đầu thì có thể 'các anh cứ làm còn tôi đứng chỉ tay'
Link: https://vietnamnet.vn/khong-co-kpi-cho-nguoi-dung-dau-thi-co-the-cac-anh-cu-lam-con-toi-dung-chi-tay-2487197.html
Published: Mon, 02 Feb 2026 07:51:00 GMT
Published parsed: time.struct_time(tm_year=2026, tm_mon=2, tm_mday=2, tm_hour=7, tm_min=51, tm_sec=0, tm_wday=0, tm_yday=33, tm_isdst=0)
Author: Trần Thường

Summary length: 501 chars
Summary preview: <div><img src="https://static-images.vnncdn.net/vps_images_publish/000001/000003/2026/2/2/khong-co-kpi-cho-nguoi-dung-dau-thi-co-the-cac-anh-cu-lam-con-toi-dung-chi-tay-1715.jpg?width=0&amp;s=te330VQjU7iFsqDcE7SUwA" style="width: 100%;" /><div>Ông Nguyễn Xuân Thắng cho rằng, trong công vi

In [7]:
# Thống kê nguồn tin
import pandas as pd

df = pd.read_csv("rss_feed_articles.csv")

print(f"Tổng số bài viết: {len(df)}")
print(f"\nThống kê theo nguồn:")
print(df['source.name'].value_counts())

print(f"\nThống kê theo ngày:")
df['date'] = pd.to_datetime(df['published_at']).dt.date
print(df['date'].value_counts().sort_index(ascending=False).head(10))

print(f"\nMẫu 3 bài viết:")
print(df[['title', 'source.name', 'published_at']].head(3).to_string(index=False))

Tổng số bài viết: 729

Thống kê theo nguồn:
source.name
RSS_Feed                                        504
Thành Đạt                                        28
Minh Phương                                      27
Đức Hoàng                                        25
Tri Túc                                          21
Khổng Chiêm                                       9
Thanh Thương                                      9
Thảo Thu                                          9
Toàn Thịnh                                        9
Mỹ Tâm                                            8
Thanh Thành                                       8
Huỳnh Anh                                         8
Cẩm Hà                                            8
VnExpress                                         6
N. Tuấn Sơn                                       5
Minh Huyền                                        5
Đăng Khôi                                         4
Trường Thịnh                                      4
CTV     

In [8]:
# Kiểm tra dữ liệu keywords và category.primary hiện tại
import pandas as pd

df = pd.read_csv("rss_feed_articles.csv")

print("=== KIỂM TRA DỮ LIỆU HIỆN TẠI ===\n")
print(f"1. Keywords:")
print(f"   - Số bài có keywords: {df['keywords'].notna().sum()}/{len(df)}")
print(f"   - Mẫu keywords (nếu có):")
print(df[df['keywords'].notna()]['keywords'].head(3).tolist())

print(f"\n2. Category.primary:")
print(f"   - Số bài có category: {df['category.primary'].notna().sum()}/{len(df)}")
print(f"   - Mẫu category (nếu có):")
print(df[df['category.primary'].notna()]['category.primary'].value_counts().head(10))

print(f"\n3. Phân tích URL để tìm category:")
df['url_path'] = df['url'].str.replace(r'https?://', '', regex=True).str.split('/').str[1:3]
sample_urls = df[['url', 'url_path']].head(10)
print(sample_urls.to_string(index=False))

=== KIỂM TRA DỮ LIỆU HIỆN TẠI ===

1. Keywords:
   - Số bài có keywords: 200/729
   - Mẫu keywords (nếu có):
['Thế giới', 'Thế giới', 'Thế giới']

2. Category.primary:
   - Số bài có category: 200/729
   - Mẫu category (nếu có):
category.primary
Thế giới      100
Kinh doanh    100
Name: count, dtype: int64

3. Phân tích URL để tìm category:
                                                                                                                                     url                                                                                                             url_path
                       https://vietnamnet.vn/khong-co-kpi-cho-nguoi-dung-dau-thi-co-the-cac-anh-cu-lam-con-toi-dung-chi-tay-2487197.html                        [khong-co-kpi-cho-nguoi-dung-dau-thi-co-the-cac-anh-cu-lam-con-toi-dung-chi-tay-2487197.html]
                   https://laodong.vn/du-lich/cong-dong/du-bao-thoi-tiet-mua-ret-khong-khi-lanh-tu-nay-den-tet-am-lich-2026-1649823.html             

In [9]:
# Kiểm tra thông tin category/tags từ RSS feeds
import feedparser

print("=== PHÂN TÍCH THÔNG TIN CATEGORY/TAGS TỪ RSS FEEDS ===\n")

# Test với 3 RSS feeds khác nhau
test_feeds = [
    ("VnExpress Thời sự", "https://vnexpress.net/rss/thoi-su.rss"),
    ("Dân Trí Xã hội", "https://dantri.com.vn/rss/xa-hoi.rss"),
    ("Thanh Niên Home", "https://thanhnien.vn/rss/home.rss"),
]

for name, url in test_feeds:
    print(f"\n{'='*60}")
    print(f"Feed: {name}")
    print(f"URL: {url}")
    print(f"{'='*60}")
    
    try:
        feed = feedparser.parse(url)
        
        if feed.entries:
            entry = feed.entries[0]
            print(f"\nBài viết mẫu: {entry.get('title', 'N/A')[:80]}...")
            print(f"\nCác trường có sẵn trong entry:")
            
            # Kiểm tra các trường có thể chứa category/tags
            fields_to_check = ['tags', 'categories', 'category', 'keywords', 
                             'dc_subject', 'subjects', 'description']
            
            for field in fields_to_check:
                if field in entry:
                    value = entry[field]
                    print(f"  ✓ {field}: {value}")
            
            # Hiển thị tất cả keys để xem còn gì khác
            print(f"\n  Tất cả keys: {list(entry.keys())}")
            
    except Exception as e:
        print(f"  ✗ Lỗi: {e}")

=== PHÂN TÍCH THÔNG TIN CATEGORY/TAGS TỪ RSS FEEDS ===


Feed: VnExpress Thời sự
URL: https://vnexpress.net/rss/thoi-su.rss

Bài viết mẫu: Vợ chồng bị phóng điện khi dựng cây nêu ngày Tết...

Các trường có sẵn trong entry:
  ✓ description: <a href="https://vnexpress.net/vo-chong-bi-phong-dien-khi-dung-cay-neu-ngay-tet-5013260.html"><img src="https://i1-vnexpress.vnecdn.net/2026/02/02/cay-neu-1770031368-1770031378-5033-1770031664.jpg?w=1200&amp;h=0&amp;q=100&amp;dpr=1&amp;fit=crop&amp;s=aIenNgbvud9CbTqrcJBwAg" /></a>Cặp vợ chồng ở xã Nhân Hòa bị phóng điện khi dựng cây nêu tre vướng đường dây 35 kV, người vợ bị bỏng nặng, nguy cơ phải cắt cụt chi.

  Tất cả keys: ['title', 'title_detail', 'summary', 'summary_detail', 'published', 'published_parsed', 'links', 'link', 'id', 'guidislink']

Feed: Dân Trí Xã hội
URL: https://dantri.com.vn/rss/xa-hoi.rss

Bài viết mẫu: Hai vụ cháy trong một buổi chiều ở Nghệ An...

Các trường có sẵn trong entry:
  ✓ tags: [{'term': 'Thời sự', 'scheme': 'https

In [10]:
# Test trích xuất category và keywords
import re
from urllib.parse import urlparse

def extract_category_from_url(url):
    """Trích xuất category từ URL pattern"""
    
    # Mapping các patterns phổ biến
    category_map = {
        'thoi-su': 'Thời sự',
        'the-gioi': 'Thế giới', 
        'xa-hoi': 'Xã hội',
        'kinh-doanh': 'Kinh doanh',
        'giai-tri': 'Giải trí',
        'the-thao': 'Thể thao',
        'phap-luat': 'Pháp luật',
        'giao-duc': 'Giáo dục',
        'suc-khoe': 'Sức khỏe',
        'gia-dinh': 'Gia đình',
        'du-lich': 'Du lịch',
        'khoa-hoc': 'Khoa học',
        'so-hoa': 'Số hóa',
        'cong-nghe': 'Công nghệ',
        'oto-xe-may': 'Ôtô-Xe máy',
        'doi-song': 'Đời sống',
        'van-hoa': 'Văn hóa',
        'tin-tuc-trong-ngay': 'Tin tức',
        'video': 'Video',
    }
    
    parsed = urlparse(url)
    path_parts = [p for p in parsed.path.split('/') if p]
    
    # Tìm category trong các phần của URL
    for part in path_parts[:2]:  # Chỉ xét 2 phần đầu
        for pattern, category in category_map.items():
            if pattern in part:
                return category
    
    return None

def extract_keywords_from_rss(entry, url):
    """Trích xuất keywords từ RSS entry và URL"""
    keywords = []
    
    # 1. Lấy từ tags RSS (Dân Trí, Tuổi Trẻ)
    if 'tags' in entry:
        for tag in entry['tags']:
            if 'term' in tag and tag['term']:
                keywords.append(tag['term'])
    
    # 2. Lấy từ category field
    if 'category' in entry and entry['category']:
        keywords.append(entry['category'])
    
    # 3. Trích xuất từ URL
    url_category = extract_category_from_url(url)
    if url_category and url_category not in keywords:
        keywords.append(url_category)
    
    return ', '.join(keywords) if keywords else ''

# Test với URLs mẫu
test_urls = [
    "https://vnexpress.net/thoi-su/chinh-tri/bai-viet-123.html",
    "https://dantri.com.vn/the-gioi/my-trung-456.htm",
    "https://thanhnien.vn/kinh-doanh/chung-khoan-789.htm",
    "https://tuoitre.vn/giao-duc/dai-hoc-101.htm",
]

print("=== TEST TRÍCH XUẤT CATEGORY TỪ URL ===\n")
for url in test_urls:
    category = extract_category_from_url(url)
    print(f"URL: {url}")
    print(f"→ Category: {category}\n")

=== TEST TRÍCH XUẤT CATEGORY TỪ URL ===

URL: https://vnexpress.net/thoi-su/chinh-tri/bai-viet-123.html
→ Category: Thời sự

URL: https://dantri.com.vn/the-gioi/my-trung-456.htm
→ Category: Thế giới

URL: https://thanhnien.vn/kinh-doanh/chung-khoan-789.htm
→ Category: Kinh doanh

URL: https://tuoitre.vn/giao-duc/dai-hoc-101.htm
→ Category: Giáo dục



In [11]:
# CẬP NHẬT: Thêm tự động tạo keywords và category vào crawler chính

# Backup file hiện tại
import shutil
shutil.copy("rss_feed_articles.csv", "rss_feed_articles_backup.csv")
print("✓ Đã backup file CSV hiện tại")

# Xóa file cũ để crawl lại với keywords và category
import os
if os.path.exists("rss_feed_articles.csv"):
    os.remove("rss_feed_articles.csv")
print("✓ Đã xóa file cũ, sẵn sàng crawl lại với keywords/category tự động")

print("\n⚠️ LƯU Ý: Bạn cần chạy lại cell #VSC-6bad2b57 với code đã cập nhật")
print("Hoặc tôi có thể tạo một file crawler mới với đầy đủ tính năng.")

✓ Đã backup file CSV hiện tại
✓ Đã xóa file cũ, sẵn sàng crawl lại với keywords/category tự động

⚠️ LƯU Ý: Bạn cần chạy lại cell #VSC-6bad2b57 với code đã cập nhật
Hoặc tôi có thể tạo một file crawler mới với đầy đủ tính năng.


In [14]:
# Kiểm tra kết quả sau khi crawl lại với keywords và category
import pandas as pd

df = pd.read_csv("rss_feed_articles.csv")

print("=" * 80)
print("KẾT QUẢ SAU KHI CẬP NHẬT KEYWORDS VÀ CATEGORY")
print("=" * 80)

print(f"\nTổng số bài viết: {len(df)}")

print(f"\n1. KEYWORDS:")
print(f"   - Số bài có keywords: {df['keywords'].notna().sum()}/{len(df)} ({df['keywords'].notna().sum()/len(df)*100:.1f}%)")
keywords_sample = df[df['keywords'].notna()]['keywords'].head(5)
print(f"   - Mẫu keywords:")
for i, kw in enumerate(keywords_sample, 1):
    print(f"     {i}. {kw[:100]}")

print(f"\n2. CATEGORY.PRIMARY:")
print(f"   - Số bài có category: {df['category.primary'].notna().sum()}/{len(df)} ({df['category.primary'].notna().sum()/len(df)*100:.1f}%)")
print(f"   - Top 10 categories:")
print(df['category.primary'].value_counts().head(10))

print(f"\n3. MẪU 5 BÀI VIẾT:")
sample_df = df[['title', 'category.primary', 'keywords']].head(5)
for idx, row in sample_df.iterrows():
    print(f"\n   [{idx+1}] {row['title'][:60]}...")
    print(f"       Category: {row['category.primary'] if pd.notna(row['category.primary']) else 'N/A'}")
    keywords_str = row['keywords'] if pd.notna(row['keywords']) else 'N/A'
    print(f"       Keywords: {str(keywords_str)[:80]}")

KẾT QUẢ SAU KHI CẬP NHẬT KEYWORDS VÀ CATEGORY

Tổng số bài viết: 728

1. KEYWORDS:
   - Số bài có keywords: 248/728 (34.1%)
   - Mẫu keywords:
     1. Du lịch
     2. Tin tức
     3. Thời sự
     4. Thời sự
     5. Thời sự

2. CATEGORY.PRIMARY:
   - Số bài có category: 248/728 (34.1%)
   - Top 10 categories:
category.primary
Thế giới      116
Kinh doanh    102
Du lịch         7
Giải trí        4
Thời sự         3
Xã hội          3
Đời sống        2
Tin tức         2
Sức khỏe        2
Công nghệ       1
Name: count, dtype: int64

3. MẪU 5 BÀI VIẾT:

   [1] Việt Nam sẽ giúp Cuba từng bước tự chủ lương thực...
       Category: N/A
       Keywords: N/A

   [2] Thủ khoa thi đánh giá tư duy Bách khoa Hà Nội đạt 96,1/100 đ...
       Category: N/A
       Keywords: N/A

   [3] Tin mới vụ nhóm du khách nước ngoài bị đuổi đánh ở Nha Trang...
       Category: N/A
       Keywords: N/A

   [4] Không có KPI cho người đứng đầu thì có thể 'các anh cứ làm c...
       Category: N/A
       Keywords: N/A

 

In [15]:
# Phân tích các bài viết không có keywords/category
import pandas as pd

df = pd.read_csv("rss_feed_articles.csv")

# Lấy các bài không có category
no_cat = df[df['category.primary'].isna()]

print(f"Số bài không có category: {len(no_cat)}")
print(f"\nPhân tích nguồn của các bài không có category:")
print(no_cat['source.name'].value_counts().head(10))

print(f"\nMẫu 10 URLs của bài không có category:")
for i, url in enumerate(no_cat['url'].head(10), 1):
    print(f"{i}. {url}")

Số bài không có category: 480

Phân tích nguồn của các bài không có category:
source.name
RSS_Feed                            461
VnExpress                             7
Trần Thường                           2
Ca Linh-Tâm Minh                      1
Znews.vn                              1
VietNamNet News                       1
QU&#7888;C NAM                        1
Lã Nghĩa Hiếu                         1
https://www.facebook.com/bbcnews      1
THÀNH CHUNG                           1
Name: count, dtype: int64

Mẫu 10 URLs của bài không có category:
1. https://vnexpress.net/viet-nam-se-giup-cuba-tung-buoc-tu-chu-luong-thuc-5013267.html
2. https://vnexpress.net/thu-khoa-thi-danh-gia-tu-duy-bach-khoa-ha-noi-dat-96-1-100-diem-5012894.html
3. https://znews.vn/tin-moi-vu-nhom-du-khach-nuoc-ngoai-bi-duoi-danh-o-nha-trang-post1616048.html
4. https://vietnamnet.vn/khong-co-kpi-cho-nguoi-dung-dau-thi-co-the-cac-anh-cu-lam-con-toi-dung-chi-tay-2487197.html
5. https://nld.com.vn/ket-qua-dieu-tra-

In [16]:
# Test trích xuất category từ các URL thực tế
test_real_urls = [
    "https://vnexpress.net/viet-nam-se-giup-cuba-tung-buoc-tu-chu-luong-thuc-5013267.html",
    "https://vietnamnet.vn/khong-co-kpi-cho-nguoi-dung-dau-thi-co-the-cac-anh-cu-lam-con-toi-dung-chi-tay-2487197.html",
    "https://tuoitre.vn/23-hoc-sinh-bi-cong-an-moi-lam-viec-vi-ra-duong-nhay-mua-theo-tieng-coi-xe-tai-20260202181003099.htm",
    "https://nld.com.vn/ket-qua-dieu-tra-ban-dau-vu-2-nguoi-tu-vong-sau-khi-uong-ruou-ngam-196260202160814492.htm",
]

print("Test trích xuất category từ URLs không có category trong path:\n")
for url in test_real_urls:
    cat = extract_category_from_url(url)
    print(f"{cat or 'NONE':<15} | {url}")

Test trích xuất category từ URLs không có category trong path:

NONE            | https://vnexpress.net/viet-nam-se-giup-cuba-tung-buoc-tu-chu-luong-thuc-5013267.html
NONE            | https://vietnamnet.vn/khong-co-kpi-cho-nguoi-dung-dau-thi-co-the-cac-anh-cu-lam-con-toi-dung-chi-tay-2487197.html
NONE            | https://tuoitre.vn/23-hoc-sinh-bi-cong-an-moi-lam-viec-vi-ra-duong-nhay-mua-theo-tieng-coi-xe-tai-20260202181003099.htm
NONE            | https://nld.com.vn/ket-qua-dieu-tra-ban-dau-vu-2-nguoi-tu-vong-sau-khi-uong-ruou-ngam-196260202160814492.htm


In [17]:
print("=" * 80)
print("TÓM TẮT KẾT QUẢ")
print("=" * 80)

print("""
✅ ĐÃ HOÀN THÀNH:
1. Tự động trích xuất keywords và category từ RSS feeds
2. Thu thập được 728 bài viết từ 17 RSS feeds
3. Có 248/728 bài (34.1%) đã có keywords và category tự động

📊 KẾT QUẢ CHI TIẾT:
- Keywords: 34.1% bài viết có keywords
- Category: 34.1% bài viết có category
- Top categories: Thế giới (116), Kinh doanh (102), Du lịch (7)

🔍 PHƯƠNG PHÁP TRÍCH XUẤT:
1. Từ RSS tags (Dân Trí, Tuổi Trẻ có sẵn tags)
2. Từ URL pattern (nhận diện /thoi-su/, /the-gioi/, etc.)
3. Ưu tiên: RSS tags > URL category

⚠️ HẠN CHẾ:
- 66% bài viết không có category do:
  + RSS feed Google News không có tags
  + Một số URL không chứa category trong path
  + VnExpress, Thanh Niên không cung cấp tags trong RSS

💡 CẢI THIỆN THÊM (NẾU CẦN):
1. Sử dụng NLP để phân loại tự động từ title/content
2. Trích xuất keywords từ title bằng TF-IDF
3. Fetch metadata từ trang web gốc (trong thẻ <meta>)
4. Training model phân loại category dựa trên nội dung

🎯 KHUYẾN NGHỊ:
- Nếu chỉ cần keywords/category cho phân tích cơ bản: Đã đủ (34% có data)
- Nếu cần 100% bài có category: Cần thêm NLP hoặc fetch metadata từ web
- Có thể filter chỉ lấy bài từ RSS feeds có tags (Dân Trí, Tuổi Trẻ)
""")

TÓM TẮT KẾT QUẢ

✅ ĐÃ HOÀN THÀNH:
1. Tự động trích xuất keywords và category từ RSS feeds
2. Thu thập được 728 bài viết từ 17 RSS feeds
3. Có 248/728 bài (34.1%) đã có keywords và category tự động

📊 KẾT QUẢ CHI TIẾT:
- Keywords: 34.1% bài viết có keywords
- Category: 34.1% bài viết có category
- Top categories: Thế giới (116), Kinh doanh (102), Du lịch (7)

🔍 PHƯƠNG PHÁP TRÍCH XUẤT:
1. Từ RSS tags (Dân Trí, Tuổi Trẻ có sẵn tags)
2. Từ URL pattern (nhận diện /thoi-su/, /the-gioi/, etc.)
3. Ưu tiên: RSS tags > URL category

⚠️ HẠN CHẾ:
- 66% bài viết không có category do:
  + RSS feed Google News không có tags
  + Một số URL không chứa category trong path
  + VnExpress, Thanh Niên không cung cấp tags trong RSS

💡 CẢI THIỆN THÊM (NẾU CẦN):
1. Sử dụng NLP để phân loại tự động từ title/content
2. Trích xuất keywords từ title bằng TF-IDF
3. Fetch metadata từ trang web gốc (trong thẻ <meta>)
4. Training model phân loại category dựa trên nội dung

🎯 KHUYẾN NGHỊ:
- Nếu chỉ cần keywords/categor

In [19]:
# Hiển thị danh sách RSS feeds đã cập nhật
print("=" * 80)
print("DANH SÁCH RSS FEEDS ĐÃ CẬP NHẬT")
print("=" * 80)

print(f"\nTổng số feeds: {len(RSS_FEEDS)}")
print("\nPhân loại theo nguồn:")

sources = {}
for feed in RSS_FEEDS:
    if "vnexpress" in feed:
        source = "VnExpress"
    elif "dantri" in feed:
        source = "Dân Trí"
    elif "tuoitre" in feed:
        source = "Tuổi Trẻ"
    elif "thanhnien" in feed:
        source = "Thanh Niên"
    elif "vietnamnet" in feed:
        source = "VietnamNet"
    else:
        source = "Khác"
    
    if source not in sources:
        sources[source] = []
    sources[source].append(feed)

for source, feeds in sources.items():
    print(f"\n{source}: {len(feeds)} feeds")
    for i, feed in enumerate(feeds, 1):
        # Lấy tên category từ URL
        category = feed.split('/')[-1].replace('.rss', '').replace('-', ' ').title()
        print(f"  {i}. {category}")

print("\n" + "=" * 80)
print(f"Sẵn sàng crawl {len(RSS_FEEDS)} RSS feeds!")
print("=" * 80)

DANH SÁCH RSS FEEDS ĐÃ CẬP NHẬT

Tổng số feeds: 47

Phân loại theo nguồn:

VnExpress: 15 feeds
  1. Tin Moi Nhat
  2. Thoi Su
  3. The Gioi
  4. Kinh Doanh
  5. Giai Tri
  6. The Thao
  7. Phap Luat
  8. Giao Duc
  9. Suc Khoe
  10. Gia Dinh
  11. Du Lich
  12. Khoa Hoc
  13. So Hoa
  14. Oto Xe May
  15. Y Kien

Dân Trí: 10 feeds
  1. Trang Chu
  2. Xa Hoi
  3. The Gioi
  4. Kinh Doanh
  5. The Thao
  6. Giai Tri
  7. Giao Duc
  8. Suc Khoe
  9. Du Lich
  10. O To Xe May

Tuổi Trẻ: 9 feeds
  1. Tin Moi Nhat
  2. Thoi Su
  3. The Gioi
  4. Phap Luat
  5. Kinh Doanh
  6. Giao Duc
  7. The Thao
  8. Giai Tri
  9. Xe

Thanh Niên: 8 feeds
  1. Home
  2. Thoi Su
  3. The Gioi
  4. Kinh Te
  5. Van Hoa
  6. The Thao
  7. Cong Nghe
  8. Gioi Tre

VietnamNet: 5 feeds
  1. Thoi Su
  2. The Gioi
  3. Kinh Doanh
  4. Giao Duc
  5. The Thao

Sẵn sàng crawl 47 RSS feeds!


In [20]:
# Kiểm tra kết quả sau khi crawl với 47 RSS feeds
import pandas as pd

df = pd.read_csv("rss_feed_articles.csv")

print("=" * 80)
print("KẾT QUẢ CRAWL VỚI 47 RSS FEEDS")
print("=" * 80)

print(f"\nTổng số bài viết: {len(df):,}")

print(f"\n📊 PHÂN BỐ THEO NGUỒN:")
print(df['source.name'].value_counts().head(15))

print(f"\n📅 PHÂN BỐ THEO NGÀY:")
df['date'] = pd.to_datetime(df['published_at']).dt.date
date_dist = df['date'].value_counts().sort_index(ascending=False).head(10)
for date, count in date_dist.items():
    print(f"  {date}: {count:,} bài")

print(f"\n🏷️ KEYWORDS & CATEGORY:")
keywords_count = df['keywords'].notna().sum()
category_count = df['category.primary'].notna().sum()
print(f"  - Số bài có keywords: {keywords_count:,}/{len(df):,} ({keywords_count/len(df)*100:.1f}%)")
print(f"  - Số bài có category: {category_count:,}/{len(df):,} ({category_count/len(df)*100:.1f}%)")

print(f"\n📂 TOP 10 CATEGORIES:")
print(df['category.primary'].value_counts().head(10))

print("\n" + "=" * 80)
print(f"✅ Crawl hoàn tất! File: {CSV_PATH}")
print("=" * 80)

KẾT QUẢ CRAWL VỚI 47 RSS FEEDS

Tổng số bài viết: 2,073

📊 PHÂN BỐ THEO NGUỒN:
source.name
RSS_Feed        1848
Thành Đạt         28
Minh Phương       27
Đức Hoàng         25
Tri Túc           21
Khổng Chiêm        9
Thảo Thu           9
Thanh Thương       9
Toàn Thịnh         9
Mỹ Tâm             8
Huỳnh Anh          8
Thanh Thành        8
Cẩm Hà             8
VnExpress          7
Minh Huyền         5
Name: count, dtype: int64

📅 PHÂN BỐ THEO NGÀY:
  2026-02-02: 591 bài
  2026-02-01: 466 bài
  2026-01-31: 373 bài
  2026-01-30: 351 bài
  2026-01-29: 251 bài
  2026-01-28: 40 bài

🏷️ KEYWORDS & CATEGORY:
  - Số bài có keywords: 321/2,073 (15.5%)
  - Số bài có category: 321/2,073 (15.5%)

📂 TOP 10 CATEGORIES:
category.primary
Thế giới      131
Kinh doanh    107
Du lịch        17
Giáo dục        7
Khoa học        7
Xã hội          7
Video           7
Gia đình        6
Văn hóa         5
Công nghệ       5
Name: count, dtype: int64

✅ Crawl hoàn tất! File: rss_feed_articles.csv


In [22]:
# Hiển thị danh sách RSS feeds sau khi cập nhật
print("=" * 80)
print("DANH SÁCH RSS FEEDS SAU KHI CẬP NHẬT")
print("=" * 80)

print(f"\nTổng số feeds: {len(RSS_FEEDS)}")
print("\nPhân loại theo nguồn:")

sources = {}
for feed in RSS_FEEDS:
    if "vnexpress" in feed:
        source = "VnExpress"
    elif "dantri" in feed:
        source = "Dân Trí"
    elif "tuoitre" in feed:
        source = "Tuổi Trẻ"
    elif "thanhnien" in feed:
        source = "Thanh Niên"
    elif "vietnamnet" in feed:
        source = "VietnamNet"
    elif "laodong" in feed:
        source = "Lao Động"
    elif "nld.com.vn" in feed:
        source = "Người Lao Động"
    elif "vietnamplus" in feed:
        source = "VietnamPlus"
    elif "soha" in feed:
        source = "Soha"
    else:
        source = "Khác"
    
    if source not in sources:
        sources[source] = []
    sources[source].append(feed)

for source, feeds in sorted(sources.items()):
    print(f"\n{source}: {len(feeds)} feeds")

print("\n" + "=" * 80)
print(f"✅ Tổng cộng {len(RSS_FEEDS)} RSS feeds từ {len(sources)} nguồn tin!")
print("=" * 80)

print("\nLưu ý: ZNews URLs bạn cung cấp là trang HTML, không phải RSS.")
print("Nếu cần crawl ZNews, có thể cần phương pháp khác (HTML parsing).")

DANH SÁCH RSS FEEDS SAU KHI CẬP NHẬT

Tổng số feeds: 103

Phân loại theo nguồn:

Dân Trí: 10 feeds

Lao Động: 11 feeds

Người Lao Động: 17 feeds

Soha: 9 feeds

Thanh Niên: 8 feeds

Tuổi Trẻ: 9 feeds

VietnamNet: 5 feeds

VietnamPlus: 19 feeds

VnExpress: 15 feeds

✅ Tổng cộng 103 RSS feeds từ 9 nguồn tin!

Lưu ý: ZNews URLs bạn cung cấp là trang HTML, không phải RSS.
Nếu cần crawl ZNews, có thể cần phương pháp khác (HTML parsing).


In [23]:
# Kiểm tra kết quả cuối cùng
import pandas as pd

df = pd.read_csv("rss_feed_articles.csv")

print("=" * 80)
print("KẾT QUẢ CRAWL VỚI 103 RSS FEEDS")
print("=" * 80)

print(f"\n📊 TỔNG QUAN:")
print(f"   - Tổng số bài viết: {len(df):,}")
print(f"   - Số feeds: 103 (từ 9 nguồn tin)")

print(f"\n📰 PHÂN BỐ THEO NGUỒN (Top 15):")
source_dist = df['source.name'].value_counts().head(15)
for source, count in source_dist.items():
    print(f"   {source:<25} {count:>5,} bài")

print(f"\n📅 PHÂN BỐ THEO NGÀY:")
df['date'] = pd.to_datetime(df['published_at']).dt.date
date_dist = df['date'].value_counts().sort_index(ascending=False).head(7)
for date, count in date_dist.items():
    print(f"   {date}: {count:>4,} bài")

print(f"\n🏷️ KEYWORDS & CATEGORY:")
keywords_count = df['keywords'].notna().sum()
category_count = df['category.primary'].notna().sum()
print(f"   - Số bài có keywords: {keywords_count:,}/{len(df):,} ({keywords_count/len(df)*100:.1f}%)")
print(f"   - Số bài có category: {category_count:,}/{len(df):,} ({category_count/len(df)*100:.1f}%)")

print(f"\n📂 TOP 15 CATEGORIES:")
cat_dist = df['category.primary'].value_counts().head(15)
for cat, count in cat_dist.items():
    print(f"   {cat:<20} {count:>4,} bài")

# Tính độ phủ dữ liệu
avg_content_length = df['content.text'].str.len().mean()
print(f"\n📝 CHẤT LƯỢNG DỮ LIỆU:")
print(f"   - Độ dài trung bình content: {avg_content_length:,.0f} ký tự")
print(f"   - Số bài có content: {df['content.text'].notna().sum():,}/{len(df):,}")

print("\n" + "=" * 80)
print(f"✅ HOÀN TẤT! File: rss_feed_articles.csv")
print(f"   Dung lượng: {os.path.getsize('rss_feed_articles.csv') / 1024 / 1024:.2f} MB")
print("=" * 80)

KẾT QUẢ CRAWL VỚI 103 RSS FEEDS

📊 TỔNG QUAN:
   - Tổng số bài viết: 3,386
   - Số feeds: 103 (từ 9 nguồn tin)

📰 PHÂN BỐ THEO NGUỒN (Top 15):
   RSS_Feed                  3,161 bài
   Thành Đạt                    28 bài
   Minh Phương                  27 bài
   Đức Hoàng                    25 bài
   Tri Túc                      21 bài
   Khổng Chiêm                   9 bài
   Thảo Thu                      9 bài
   Thanh Thương                  9 bài
   Toàn Thịnh                    9 bài
   Mỹ Tâm                        8 bài
   Huỳnh Anh                     8 bài
   Thanh Thành                   8 bài
   Cẩm Hà                        8 bài
   VnExpress                     7 bài
   Minh Huyền                    5 bài

📅 PHÂN BỐ THEO NGÀY:
   2026-02-02: 1,103 bài
   2026-02-01:  726 bài
   2026-01-31:  551 bài
   2026-01-30:  540 bài
   2026-01-29:  406 bài
   2026-01-28:   59 bài

🏷️ KEYWORDS & CATEGORY:
   - Số bài có keywords: 856/3,386 (25.3%)
   - Số bài có category: 856/3,386 (2

In [25]:
# Hiển thị danh sách RSS feeds mới nhất
print("=" * 80)
print("DANH SÁCH RSS FEEDS MỚI NHẤT")
print("=" * 80)

print(f"\nTổng số feeds: {len(RSS_FEEDS)}")
print("\nPhân loại theo nguồn:")

sources = {}
for feed in RSS_FEEDS:
    if "vnexpress" in feed:
        source = "VnExpress"
    elif "dantri" in feed:
        source = "Dân Trí"
    elif "tuoitre" in feed:
        source = "Tuổi Trẻ"
    elif "thanhnien" in feed:
        source = "Thanh Niên"
    elif "vietnamnet" in feed:
        source = "VietnamNet"
    elif "laodong" in feed:
        source = "Lao Động"
    elif "nld.com.vn" in feed:
        source = "Người Lao Động"
    elif "vietnamplus" in feed:
        source = "VietnamPlus"
    elif "soha" in feed:
        source = "Soha"
    elif "nhandan" in feed:
        source = "Nhân Dân"
    elif "baotintuc" in feed:
        source = "Báo Tin Tức"
    elif "kienthuc" in feed:
        source = "Kiến Thức"
    else:
        source = "Khác"
    
    if source not in sources:
        sources[source] = []
    sources[source].append(feed)

for source, feeds in sorted(sources.items()):
    print(f"  {source:<20} {len(feeds):>3} feeds")

print("\n" + "=" * 80)
print(f"🎯 Tổng cộng {len(RSS_FEEDS)} RSS feeds từ {len(sources)} nguồn tin lớn!")
print("=" * 80)

DANH SÁCH RSS FEEDS MỚI NHẤT

Tổng số feeds: 137

Phân loại theo nguồn:
  Báo Tin Tức           10 feeds
  Dân Trí               10 feeds
  Kiến Thức              8 feeds
  Lao Động              11 feeds
  Người Lao Động        17 feeds
  Nhân Dân              16 feeds
  Soha                   9 feeds
  Thanh Niên             8 feeds
  Tuổi Trẻ               9 feeds
  VietnamNet             5 feeds
  VietnamPlus           19 feeds
  VnExpress             15 feeds

🎯 Tổng cộng 137 RSS feeds từ 12 nguồn tin lớn!


In [26]:
# Kiểm tra kết quả sau khi crawl 137 RSS feeds
import pandas as pd

df = pd.read_csv("rss_feed_articles.csv")

print("=" * 80)
print("KẾT QUẢ CRAWL VỚI 137 RSS FEEDS")
print("=" * 80)

print(f"\n📊 TỔNG QUAN:")
print(f"   - Tổng số bài viết: {len(df):,}")
print(f"   - Số feeds: 137 (từ 12 nguồn tin)")
print(f"   - Tăng từ 3,386 → {len(df):,} bài ({(len(df)-3386)/3386*100:+.1f}%)")

print(f"\n📰 PHÂN BỐ THEO NGUỒN (Top 20):")
source_dist = df['source.name'].value_counts().head(20)
for source, count in source_dist.items():
    print(f"   {source:<30} {count:>5,} bài")

print(f"\n📅 PHÂN BỐ THEO NGÀY:")
df['date'] = pd.to_datetime(df['published_at']).dt.date
date_dist = df['date'].value_counts().sort_index(ascending=False).head(7)
for date, count in date_dist.items():
    print(f"   {date}: {count:>4,} bài")

print(f"\n🏷️ KEYWORDS & CATEGORY:")
keywords_count = df['keywords'].notna().sum()
category_count = df['category.primary'].notna().sum()
print(f"   - Số bài có keywords: {keywords_count:,}/{len(df):,} ({keywords_count/len(df)*100:.1f}%)")
print(f"   - Số bài có category: {category_count:,}/{len(df):,} ({category_count/len(df)*100:.1f}%)")

print(f"\n📂 TOP 20 CATEGORIES:")
cat_dist = df['category.primary'].value_counts().head(20)
for cat, count in cat_dist.items():
    print(f"   {cat:<25} {count:>4,} bài")

# Tính độ phủ dữ liệu
avg_content_length = df['content.text'].str.len().mean()
print(f"\n📝 CHẤT LƯỢNG DỮ LIỆU:")
print(f"   - Độ dài trung bình content: {avg_content_length:,.0f} ký tự")
print(f"   - Số bài có content: {df['content.text'].notna().sum():,}/{len(df):,}")

print("\n" + "=" * 80)
print(f"✅ HOÀN TẤT! File: rss_feed_articles.csv")
file_size_mb = os.path.getsize('rss_feed_articles.csv') / 1024 / 1024
print(f"   Dung lượng: {file_size_mb:.2f} MB")
print(f"   Nguồn tin: 12 tờ báo lớn nhất Việt Nam")
print("=" * 80)

KẾT QUẢ CRAWL VỚI 137 RSS FEEDS

📊 TỔNG QUAN:
   - Tổng số bài viết: 4,027
   - Số feeds: 137 (từ 12 nguồn tin)
   - Tăng từ 3,386 → 4,027 bài (+18.9%)

📰 PHÂN BỐ THEO NGUỒN (Top 20):
   RSS_Feed                       3,802 bài
   Thành Đạt                         28 bài
   Minh Phương                       27 bài
   Đức Hoàng                         25 bài
   Tri Túc                           21 bài
   Khổng Chiêm                        9 bài
   Thảo Thu                           9 bài
   Thanh Thương                       9 bài
   Toàn Thịnh                         9 bài
   Mỹ Tâm                             8 bài
   Huỳnh Anh                          8 bài
   Thanh Thành                        8 bài
   Cẩm Hà                             8 bài
   VnExpress                          7 bài
   Minh Huyền                         5 bài
   N. Tuấn Sơn                        5 bài
   Trường Thịnh                       4 bài
   Đăng Khôi                          4 bài
   Trần Thường          